In [2]:
import pandas as pd
import re

df = pd.read_csv('merged.csv')

df['id'] = range(1, len(df) + 1)

text_columns = ['категория', 'название вакансии', 'адрес', 'город', 'регион', 'страна',
                'место работы', 'график работы', 'образование', 'кандидат', 'зарплата']
for col in text_columns:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().str.lower()

if 'дата_размещения' in df.columns:
    df['дата_размещения'] = pd.to_datetime(df['дата_размещения'], errors='coerce')

def parse_salary_and_experience(text):
    if pd.isna(text):
        return None, None, None

    s = str(text).replace('\u202f', '').replace('\xa0', '').lower()

    exp = None
    if 'без опыта' in s or 'не требуется опыт' in s:
        exp = '0'
    elif re.match(r'^\s*(опыт|без)', s):
        nums = re.findall(r'\d+', s)
        if nums:
            if re.search(r'более|от', s):
                exp = nums[0]
            elif re.search(r'до', s):
                exp = nums[0]
            elif len(nums) >= 2:
                exp = f"{nums[0]}-{nums[1]}"
            else:
                exp = nums[0]
        else:
            exp = None
    else:
        parts = re.split(r'опыт', s, maxsplit=1, flags=re.IGNORECASE)
        exp_part = parts[1] if len(parts) > 1 else ''
        if exp_part:
            nums = re.findall(r'\d+', exp_part)
            if nums:
                if re.search(r'более|от', exp_part):
                    exp = nums[0]
                elif re.search(r'до', exp_part):
                    exp = nums[0]
                elif len(nums) >= 2:
                    exp = f"{nums[0]}-{nums[1]}"
                else:
                    exp = nums[0]
            else:
                exp = None
        else:
            exp = None

    s_no_exp = re.sub(r'опыт\s*(?:от\s*\d+\s*(?:-|до)\s*\d+|\d+\s*(?:-|до)\s*\d+|от\s*\d+|\d+)?', '', s)

    s_clean = re.sub(
        r'(руб(лей|ля)?|₽|₸|\$|so\'m|br|€|за месяц|на руки|до вычета налогов|выплаты:.*|подработка|без опыта|не требуется опыт|зарплата)',
        '',
        s_no_exp
    )

    sal_from = sal_to = None
    matches = re.findall(r'от\s*(\d+)|до\s*(\d+)|(\d+)\s*[-–—]\s*(\d+)|(\d+)', s_clean)

    for m in matches:
        if m[0]:  # "от X"
            sal_from = int(m[0])
        elif m[1]:  # "до X"
            sal_to = int(m[1])
        elif m[2] and m[3]:  # "X - Y"
            sal_from, sal_to = int(m[2]), int(m[3])
            break
        elif m[4]:  # просто число
            sal_from = sal_to = int(m[4])
            break

    if sal_from is not None and sal_from < 1000:
        sal_from = None
    if sal_to is not None and sal_to < 1000:
        sal_to = None

    return sal_from, sal_to, exp

parsed = df['зарплата'].apply(parse_salary_and_experience)
df['зарплата_от'], df['зарплата_до'], df['опыт'] = zip(*parsed)

if 'город' in df.columns and 'адрес' in df.columns:
    df['город'] = df['город'].fillna(df['адрес'].str.split(',', expand=True)[0])
    df['город'] = df['город'].str.replace(r'^(город|поселок|поселение)\s+', '', regex=True)

def country_from_city(city):
    if pd.isna(city):
        return None
    city = city.strip().lower()
    if city == 'минск':
        return 'беларусь'
    if city in ['алматы', 'астана', 'нур-султан', 'шымкент', 'атырау', 'актау']:
        return 'казахстан'
    return 'россия'

df['страна'] = df['страна'].fillna(df['город'].apply(country_from_city))

fields_to_check = ['зарплата', 'название вакансии', 'место работы', 'адрес', 'график работы', 'образование', 'кандидат']
combined = pd.Series([''] * len(df), index=df.index)
for c in fields_to_check:
    if c in df.columns:
        combined += ' ' + df[c].fillna('').astype(str)

remote_pattern = r'(удал(ен|енно|ённо)?|дистанц(ионн|ия)?|можно удалённо|remote|из дома|работа из дома|work from home|home office|telecommut)'
df['удаленная_работа'] = combined.str.contains(remote_pattern, regex=True, na=False).astype(int)

print('Пропущенные значения по столбцам:')
print(df.isna().mean().sort_values(ascending=False))

print('\nСтатистика по зарплатам:')
print(df[['зарплата_от', 'зарплата_до']].describe())

print('\nРаспределение опыта:')
print(df['опыт'].value_counts(dropna=False))

print(f"\nУдаленная работа (1 = можно, 0 = нет/не указано): {df['удаленная_работа'].sum()} / {len(df)} записей")

df.to_csv('cleaned_data.csv', index=False)

/tmp/ipython-input-1722524297.py:114: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['удаленная_работа'] = combined.str.contains(remote_pattern, regex=True, na=False).astype(int)


Пропущенные значения по столбцам:
дата_размещения      0.951493
зарплата_до          0.536101
зарплата_от          0.265951
url                  0.048507
опыт                 0.048507
id                   0.000000
адрес                0.000000
город                0.000000
название вакансии    0.000000
категория            0.000000
страна               0.000000
регион               0.000000
график работы        0.000000
место работы         0.000000
образование          0.000000
кандидат             0.000000
зарплата             0.000000
удаленная_работа     0.000000
dtype: float64

Статистика по зарплатам:
        зарплата_от   зарплата_до
count  7.869000e+03  4.973000e+03
mean   1.975934e+05  3.704407e+05
std    9.519124e+05  2.296939e+06
min    1.000000e+03  1.000000e+03
25%    5.500000e+04  6.500000e+04
50%    8.000000e+04  1.000000e+05
75%    1.200000e+05  1.700000e+05
max    3.000000e+07  8.000000e+07

Распределение опыта:
опыт
1-3     4841
0       3016
3-6     1832
None     520


In [3]:
if 'город' not in df.columns:
    df['город'] = None

mask = df['город'].isna() & df['адрес'].notna()

df.loc[mask, 'город'] = (
    df.loc[mask, 'адрес']
    .astype(str)
    .str.split(',').str[0]
    .str.strip()
    .str.lower()
)

df.to_csv('cleaned_data.csv', index=False)

In [9]:
!pip install plotly

import pandas as pd
import numpy as np
import re
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

with open("cleaned_data.csv", encoding='utf-8') as f:
    lines = f.readlines()

header_line_idx = None
for i, line in enumerate(lines):
    if "id," in line and "опыт" in line:
        header_line_idx = i
        break
if header_line_idx is None:
    for i, line in enumerate(lines):
        if len(line.strip()) > 20 and line.count('"') < 4 and not line.strip().startswith('•'):
            if any(col in line for col in ['id', 'опыт', 'зарплата_от', 'название вакансии']):
                header_line_idx = i
                break

df = pd.read_csv("cleaned_data.csv", skiprows=header_line_idx, on_bad_lines='skip')

def extract_sort_key(exp_value):
    if pd.isna(exp_value):
        return float('inf')
    s = str(exp_value).strip().lower()
    if s == '0':
        return 0
    if 'более' in s or s.endswith('+'):
        nums = re.findall(r'\d+', s)
        return int(nums[-1]) + 1 if nums else 1000
    range_match = re.search(r'(\d+)\s*[-–—]\s*(\d+)', s)
    if range_match:
        return int(range_match.group(1))
    if re.fullmatch(r'\d+', s):
        return int(s)
    return float('inf')

if 'опыт' not in df.columns:
    raise ValueError("Столбец 'опыт' не найден в cleaned_data.csv")

unique_exp = df['опыт'].dropna().astype(str).unique()
sorted_exp = sorted(unique_exp, key=extract_sort_key)

df = df[df['опыт'].notna()].copy()
df['опыт'] = df['опыт'].astype(str)
df['опыт'] = pd.Categorical(df['опыт'], categories=sorted_exp, ordered=True)

df['зарплата_сред'] = df[['зарплата_от', 'зарплата_до']].mean(axis=1)
df = df.dropna(subset=['зарплата_сред'])

def map_to_number(exp_str):
    key = extract_sort_key(exp_str)
    return key if key != float('inf') else np.nan

df['опыт_число'] = df['опыт'].apply(map_to_number)
df = df.dropna(subset=['опыт_число'])

fig1 = px.scatter(
    df,
    x='опыт',
    y='зарплата_сред',
    color='опыт',
    title='Зависимость средней зарплаты от требуемого опыта работы',
    labels={'опыт': 'Требуемый опыт (лет)', 'зарплата_сред': 'Средняя зарплата (₽)'},
    category_orders={'опыт': sorted_exp},
    hover_data=['название вакансии', 'город']
)
fig1.update_layout(xaxis_title="Опыт работы", yaxis_title="Средняя зарплата (₽)")
fig1.show()

fig2 = px.box(
    df,
    x='опыт',
    y='зарплата_сред',
    title='Распределение зарплат по уровню требуемого опыта',
    labels={'опыт': 'Требуемый опыт (лет)', 'зарплата_сред': 'Средняя зарплата (₽)'},
    category_orders={'опыт': sorted_exp}
)
fig2.update_layout(xaxis_title="Опыт работы", yaxis_title="Средняя зарплата (₽)")
fig2.show()

summary = df.groupby('опыт')['зарплата_сред'].agg(
    count='count',
    mean='mean',
    median='median',
    std='std'
).round(2)
print("\nСводная таблица по опыту:")
print(summary)

corr_df = df[['опыт_число', 'зарплата_от', 'зарплата_до', 'зарплата_сред']].dropna()
corr_matrix = corr_df.corr()

fig3 = px.imshow(
    corr_matrix,
    text_auto=True,
    color_continuous_scale='RdBu_r',
    title="Корреляционная матрица"
)
fig3.update_layout(width=600, height=500)
fig3.show()

exp_counts = df['опыт'].value_counts().reindex(sorted_exp, fill_value=0)
fig4 = px.bar(
    x=exp_counts.index,
    y=exp_counts.values,
    title='Количество вакансий по требуемому опыту',
    labels={'x': 'Опыт работы', 'y': 'Количество вакансий'},
    text=exp_counts.values
)
fig4.update_layout(xaxis_title="Опыт работы", yaxis_title="Количество вакансий")
fig4.update_traces(textposition='outside')
fig4.show()


Сводная таблица по опыту:
      count       mean    median         std
опыт                                        
0      2682  192543.50   70000.0  1204340.48
1-3    3852  216845.35   95000.0   949499.00
1       109  142391.82   80000.0   524214.21
3-6    1122  569567.39  145000.0  2783258.55
3        28  133516.07   87500.0   158622.19
6        74  740135.76  190000.0  2223127.04


/tmp/ipython-input-4235826198.py:97: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [19]:
import pandas as pd
import re

df = pd.read_csv('cleaned_data.csv', encoding='utf-8')
title_col = 'название вакансии'
df[title_col] = df[title_col].astype(str)

include_patterns = [
    r'\bbackend\b',
    r'\bразработчик\b',
    r'\bdeveloper\b',
    r'\bphp-?программист\b',
    r'\b(c#|c\+\+|java|python|go|kotlin|scala|delphi|ruby|rust)\b',
    r'\b\.net\b',
    r'\bfull[-\s]?stack\b',
    r'\bdevops\b',
    r'\bmiddle\s+t-?sql\b',
    r'\bt-?sql\s+(программист|разработчик)\b',
    r'\bqa\s*(engineer|инженер|тестировщик)\b',
    r'\bmanual\s+qa\b',
    r'\bтестировщик\s+.*\b(автоматиз|api|selenium|junior\s+qa|qa-|тестирование\s+по\s+с\s+опытом\s+в\s+банке)\b',

    r'\bdata\s+(engineer|analyst|scientist)\b',
    r'\bml\s*(engineer|разработчик|инженер)\b',
    r'\bmachine\s+learning\b',
    r'\bмашинн[оы]е?\s+обучени[ея]\b',
    r'\bbi\s*(аналитик|analyst)\b',
    r'\bpower\s*bi\b',
    r'\btableau\b',
    r'\blooker\b',
    r'\b(dwh|etl|big\s+data)\b',
    r'\bспециалист\s+по\s+аналитике\s+данных\b',
    r'\bаналитик\s+данных?\b',
    r'\bаналитик\s+.*\b(sql|bi|power\s*bi|python|ml|dwh|дашборд|dash)\b',
    r'\b(sql|bi|dash|дашборд)\s+аналитик\b',
    r'\bdata\s+quality\b',
    r'\bинженер\s+по\s+данным\b',
    r'\bdata\s+инженер\b',
    r'\bэксперт\s+.*аналитик[аи]\b.*данных?\b',

    r'\bсистемный\s+аналитик\b',
    r'\bбизнес-?аналитик\b',
    r'\bbusiness\s+analyst\b',
    r'\bаналитик\s+бизнес[а-я]*\b',
    r'\bsystem\s+analyst\b',
    r'\bстарший\s+системный\s+аналитик\b',

    r'\bэкономист\s*-?\s*аналитик\b',
    r'\bаналитик\s+эконом[а-я]*\b',
    r'\bфинансов[ыо]й\s+аналитик\b',

    r'\bаналитик\s+1с\b',
    r'\b1с-аналитик\b',
    r'\bоператор\s+1с.*\b(аналитик|продаж|логистик)\b',
    r'\bспециалист\s+1с.*\b(аналитик|продукт)\b',
    r'\bконсультант\s+1с.*\b(аналитик|продукт)\b',

    r'\bпрограммист\b',
    r'\bинженер\s+.*\b(данных|аналитик|backend|bi|qa|1с)\b',
    r'\bit\s+специалист\b',
    r'\bтехнический\s+писатель\b',
    r'\bdata\s+manager\b',
]


exclude_patterns = [
    r'\bводитель\b',
    r'\bгрузчик\b',
    r'\bоператор\s+пк\b',
    r'\bввод\s+данных\b',
    r'\bадминистратор\s+баз\s+данных\b',
    r'\bмедсестра\b',
    r'\bветеринар\b',
    r'\bкассир\b',
    r'\bпродавец\b',
    r'\bкладовщик\b',
    r'\bмастер\s+.*\b(леса|производства|участка)\b',
    r'\bбухгалтер\b',
    r'\bпомощник\s+бухгалтера\b',
    r'\bдокументовед\b',
    r'\bглавный\s+специалист\s+.*\b(бюджет|план|финанс)\b',
    r'\bассистент\s+.*\b(руководител|менеджер|админ|офис)\b',
    r'\bменеджер\s+.*\b(продаж|маркет|закуп|персонал|проект|продукт)\b',
    r'\bрекрутер\b',
    r'\bhr\b',
    r'\bофис\s+менеджер\b',
    r'\bадмин\b.*\bофис\b',
    r'\bархивариус\b',
    r'\bуборщик\b',
    r'\bохранник\b',
    r'\bповар\b',
    r'\bкурьер\b',
    r'\bэлектромонтёр\b',
    r'\bсантехник\b',
    r'\bархитектор\b\s*(?!данных)',
]

include_mask = df[title_col].str.contains('|'.join(include_patterns), case=False, na=False, regex=True)
exclude_mask = df[title_col].str.contains('|'.join(exclude_patterns), case=False, na=False, regex=True)

final_mask = include_mask & (~exclude_mask)
df_filtered = df[final_mask].copy()

output_file = 'business_informatics_jobs_max.csv'
df_filtered.to_csv(output_file, index=False, encoding='utf-8')

print(f"✅ Отфильтровано {len(df_filtered)} вакансий из {len(df)}")
print(f"📁 Сохранено: {output_file}")

✅ Отфильтровано 386 вакансий из 10720
📁 Сохранено: business_informatics_jobs_max.csv


/tmp/ipython-input-2078972240.py:107: UserWarning:

This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.

/tmp/ipython-input-2078972240.py:108: UserWarning:

This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.

